## first model feature reduction

The next step is to reduce feature size.

By reducing redundancy in data which is fed into the model it is expected to converge better, since less weights have to be adjusted.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(8, (2,2), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(16, (2,2), activation='relu'))
model.add(layers.Conv2D(32, (2,2), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 8)         40        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 16)        528       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        2080      
_________________________________________________________________
flatten (Flatten)            (None, 26912)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                861216    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 863,963
Trainable params: 863,963
Non-trainable params: 0
__________________________________________________

In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', 'srp14', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
20/20 [==============================] - 3s 145ms/step - loss: 1.1003 - acc: 0.3225
Epoch 2/20
20/20 [==============================] - 2s 81ms/step - loss: 1.0968 - acc: 0.3300
Epoch 3/20
20/20 [==============================] - 2s 84ms/step - loss: 1.0899 - acc: 0.3550
Epoch 4/20
20/20 [==============================] - 2s 77ms/step - loss: 1.0914 - acc: 0.3775
Epoch 5/20
20/20 [==============================] - 1s 70ms/step - loss: 1.0838 - acc: 0.4550
Epoch 6/20
20/20 [==============================] - 1s 70ms/step - loss: 1.0662 - acc: 0.4750
Epoch 7/20
20/20 [==============================] - 1s 70ms/step - loss: 1.0409 - acc: 0.5025
Epoch 8/20
20/20 [==============================] - 2s 76ms/step - loss: 1.0341 - acc: 0.4975
Epoch 9/20
20/20 [==============================] - 1s 69ms/step - loss: 0.9576 - acc: 0.6300
Epoch 10/20
20/20 [==============================] - 2s 76ms/step - loss: 0.9285 - acc: 0.6025
Epoch 11/20
20/20 [==============================] - 1s 71

In [6]:
model.evaluate_generator(test_generator)

[0.4432835314422846, 0.82]

In [7]:
model_path = join('models', 'symbol_classifier', 'first_model_cnn.h5')
model.save(model_path)